In [1]:
import pathlib
import os
import pandas as pd

# Data Loading

In [2]:
articles_path = "/Users/a1/Desktop/News/dataset/News Articles/"
summaries_path = "/Users/a1/Desktop/News/dataset/Summaries/"
articles_dir = pathlib.Path(articles_path)
summaries_dir = pathlib.Path(summaries_path)

In [3]:
articles_pathlist = list(articles_dir.iterdir())
summaries_pathlist = list(summaries_dir.iterdir())

In [4]:
def generate_data(folder):
    data = []
    count = 0
    for file in os.listdir(folder):
        try:
            text = ''
            name = file
            myfile = open(str(folder) + '/' + file, "r")
            text = myfile.read()
            #mylist = [name, text]
            count += 1
            data.append(text)
        except:
            continue
    print("Task Finished!")
    print(str(count) + " text files")
    
    return data, count

In [5]:
articles = []
summaries = []

for e in articles_pathlist:
    data, count = generate_data(e)
    articles.extend(data)

for e in summaries_pathlist:
    data, count = generate_data(e)
    summaries.extend(data)

Task Finished!
386 text files
Task Finished!
510 text files
Task Finished!
510 text files
Task Finished!
417 text files
Task Finished!
401 text files
Task Finished!
386 text files
Task Finished!
510 text files
Task Finished!
510 text files
Task Finished!
417 text files
Task Finished!
401 text files


In [6]:
print('Articles # : ' + str(len(articles)))
print('Summaries # : ' + str(len(summaries)))

Articles # : 2224
Summaries # : 2224


# Word Embedding

In [9]:
import re
import tensorflow as tf
import tensorflow_hub as hub

In [10]:
def preprocess_sentence(w):
    # Replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,]+", " ", w) 
    # Reducing spaces
    w = re.sub(r'[" "]+', " ", w)
    # lower case
    w = w.lower()
    return w

In [11]:
for i in range(len(articles)):
    articles[i] = preprocess_sentence(articles[i])
    
for i in range(len(summaries)):
    summaries[i] = preprocess_sentence(summaries[i])

In [12]:
len(articles)
len(summaries)

2224

In [46]:
import pandas as pd
articles = pd.DataFrame(articles)
summaries = pd.DataFrame(summaries)
data = pd.concat([articles, summaries], axis=1)

In [48]:
data.columns = ['articles', 'summaries']

In [49]:
data[:5]

,articles,summaries
0,musicians to tackle us red tape musicians grou...,nigel mccune from the musicians union said bri...
1,"u s desire to be number one u , who have won t...",but they still want more.they have to want to ...
2,rocker doherty in on stage fight rock singer p...,"babyshambles, which he formed after his acrimo..."
3,snicket tops us box office chart the film adap...,a series of unfortunate events also stars scot...
4,ocean s twelve raids box office ocean s twelve...,"ocean s twelve, the crime caper sequel starrin..."


In [53]:
data['summaries'] = data['summaries'].apply(lambda x : '_START_' + x + '_END_')

In [54]:
print("Articles : ", data['articles'][0])
print("Summaries : ", data['summaries'][0])

Articles :  musicians to tackle us red tape musicians groups are to tackle us visa regulations which are blamed for hindering british acts chances of succeeding across the atlantic. a singer hoping to perform in the us can expect to pay , simply for obtaining a visa. groups including the musicians union are calling for an end to the raw deal faced by british performers. us acts are not faced with comparable expense and bureaucracy when visiting the uk for promotional purposes. nigel mccune from the musicians union said british musicians are disadvantaged compared to their us counterparts. a sponsor has to make a petition on their behalf, which is a form amounting to nearly pages, while musicians face tougher regulations than athletes and journalists. if you make a mistake on your form, you risk a five year ban and thus the ability to further your career, says mr mccune. the us is the world s biggest music market, which means something has to be done about the creaky bureaucracy, says m

In [55]:
articles_max_len = 500
summaries_max_len = 100

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
x_train, x_val, y_train, y_val = train_test_split(data['articles'], data['summaries'], test_size=0.1, random_state=0, shuffle=True)

In [60]:
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_train))

In [61]:
x_train = x_tokenizer.texts_to_sequences(x_train)
x_val = x_tokenizer.texts_to_sequences(x_val)

In [62]:
x_train = pad_sequences(x_train, 
                        maxlen=articles_max_len, 
                        padding='post')
x_val = pad_sequences(x_val, 
                    maxlen=articles_max_len, 
                    padding='post')

In [63]:
x_voc_size = len(x_tokenizer.word_index) + 1

In [65]:
x_train.shape

(2001, 500)

In [68]:
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_train))

In [69]:
y_train = y_tokenizer.texts_to_sequences(y_train)
y_val = y_tokenizer.texts_to_sequences(y_val)

In [71]:
y_train = pad_sequences(y_train, 
                        maxlen=summaries_max_len, 
                        padding='post')
y_val = pad_sequences(y_val, 
                    maxlen=summaries_max_len, 
                    padding='post')

In [72]:
y_voc_size = len(y_tokenizer.word_index) + 1

In [73]:
y_train.shape

(2001, 100)

In [75]:
def glove_100d_dictionary(GLOVE_DIR):
    embeddings_index = {}
    f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [76]:
embeddings_index = {}
f = open(os.path.join('/Users/a1/Desktop/', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [78]:
word_index = x_tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [80]:
print(embeddings_index[])

KeyError: 'Hello'